<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#filter-only-injured-(injury_severity-!=-0)" data-toc-modified-id="filter-only-injured-(injury_severity-!=-0)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>filter only injured (injury_severity != 0)</a></span></li><li><span><a href="#filter-ages-5-19-only" data-toc-modified-id="filter-ages-5-19-only-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>filter ages 5-19 only</a></span></li><li><span><a href="#unique-id-per-involved" data-toc-modified-id="unique-id-per-involved-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>unique id per involved</a></span></li><li><span><a href="#filter-pedastrians,-cyclists-or-electrical-vehicles" data-toc-modified-id="filter-pedastrians,-cyclists-or-electrical-vehicles-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>filter pedastrians, cyclists or electrical vehicles</a></span></li><li><span><a href="#filter-only-schools-not-kindergardens" data-toc-modified-id="filter-only-schools-not-kindergardens-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>filter only schools not kindergardens</a></span></li><li><span><a href="#check-main-roads" data-toc-modified-id="check-main-roads-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>check main roads</a></span></li><li><span><a href="#Get-values-of-past-year---June-2019---May-2020-and-calculate-formula-using-נוהל-פר״ת" data-toc-modified-id="Get-values-of-past-year---June-2019---May-2020-and-calculate-formula-using-נוהל-פר״ת-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Get values of past year - June 2019 - May 2020 and calculate formula using נוהל פר״ת</a></span></li><li><span><a href="#Compare-cities" data-toc-modified-id="Compare-cities-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Compare cities</a></span></li><li><span><a href="#load-yishuv-data" data-toc-modified-id="load-yishuv-data-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>load yishuv data</a></span></li></ul></div>

In [1]:
import pandas as pd
from collections import defaultdict

In [2]:
import os

In [3]:
li = []

for i, filename in enumerate(os.listdir('schools_data')):
    if i % 100 == 0:
        print(i)
    df = pd.read_csv(os.path.join('schools_data', filename), index_col=None, header=0)
    li.append(df)

total_df = pd.concat(li, axis=0, ignore_index=True)
total_df['accident_timestamp'] = pd.to_datetime(total_df.accident_timestamp.values)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500


In [4]:
total_df_orig = total_df.copy()

### filter only injured (injury_severity != 0)

In [5]:
total_df = total_df.loc[total_df.injury_severity != 0]

### filter ages 5-19 only

In [6]:
total_df = total_df.loc[total_df.age_group >= 2]

### unique id per involved

In [7]:
type(total_df['provider_and_id'].iloc[0])

int

In [8]:
total_df['inv_unique_id'] = total_df['provider_and_id'].astype(str) + '_' +  total_df['involve_id'].astype(str)


### filter pedastrians, cyclists or electrical vehicles

In [9]:
total_df = total_df.loc[(total_df.injured_type == 1) | (total_df.involve_vehicle_type == 21) | (total_df.involve_vehicle_type == 23) | (total_df.involve_vehicle_type == 15)]
                                                                                                 

In [10]:
total_df['vehicle_or_pedastrian'] = total_df.apply(lambda x: x['involve_vehicle_type_hebrew'] if x['injured_type_hebrew'] != 'הולך רגל' else  x['injured_type_hebrew'],
                                                  axis=1)


In [11]:
total_df['vehicle_or_pedastrian'].unique()

array(['אופניים', 'הולך רגל', 'אופניים חשמליים', 'קורקינט חשמלי'],
      dtype=object)

In [12]:
total_df.shape

(56732, 165)

### filter only schools not kindergardens

In [13]:
schools_df = total_df.loc[total_df.school_type == 'בית ספר']

In [14]:
schools_df.accident_timestamp.head()

1    2017-11-28 16:45:00
2    2018-03-19 07:45:00
8    2014-07-27 09:00:00
9    2020-05-13 16:00:00
10   2019-10-23 08:30:00
Name: accident_timestamp, dtype: datetime64[ns]

### check main roads

In [15]:
main_roads = schools_df.groupby('road1').inv_unique_id.nunique().rename('count_in_roads').to_frame()
main_roads.sort_values('count_in_roads', ascending=False).iloc[0:30]

,count_in_roads
road1,
4.0,28
40.0,13
805.0,10
1.0,9
65.0,9
20.0,8
444.0,6
44.0,6
75.0,5


In [16]:
main_roads.shape

(49, 1)

### Get values of past year - June 2019 - May 2020 and calculate formula using נוהל פר״ת

In [17]:
killed_weight = 6600/7581
severe_weight = 956/7581
light_weight = 25/7581

In [18]:
analysis = defaultdict(dict)

for school_id in schools_df.school_id.unique():
    school_df = schools_df.loc[schools_df.school_id == school_id]
    df_2019_2020 = school_df.loc[(school_df.accident_timestamp >= pd.Timestamp('2015-06-01')) & (school_df.accident_timestamp <= pd.Timestamp('2020-05-31'))]
    killed = df_2019_2020.loc[df_2019_2020.injury_severity_hebrew == 'הרוג'].shape[0]
    severe_injured = df_2019_2020.loc[df_2019_2020.injury_severity_hebrew == 'פצוע קשה'].shape[0]
    light_injured = df_2019_2020.loc[df_2019_2020.injury_severity_hebrew == 'פצוע קל'].shape[0]
    score = (killed * killed_weight + severe_injured * severe_weight  + light_injured * light_weight) * (killed + severe_injured + light_injured)
    analysis[school_id]['score'] = score
    analysis[school_id]['school_name'] = school_df.school_name.iloc[0]
    analysis[school_id]['school_yishuv_name'] = school_df.school_yishuv_name.iloc[0]
    analysis[school_id]['killed'] = killed
    analysis[school_id]['severe_injured'] = severe_injured
    analysis[school_id]['light_injured'] = light_injured
analysis = pd.DataFrame(analysis).T

In [19]:
analysis.sort_values('score', ascending=False).iloc[0:40]

,score,school_name,school_yishuv_name,killed,severe_injured,light_injured
513515,228.043,תת רוזין בני ברק,בני ברק,1,8,95
541631,220.465,ישיבת חסידי דאראג,בני ברק,1,8,92
541599,217.953,שיח יצחק,בני ברק,1,8,91
514760,211.651,שובו-מעורב,תל אביב - יפו,2,9,58
511477,166.594,קרית חינוך יפו,תל אביב - יפו,2,5,58
540179,166.594,קרית חינוך יפו,תל אביב - יפו,2,5,58
513846,151.402,כתב סופר,בני ברק,0,8,104
519629,139.612,זיו התורה,ירושלים,1,5,74
512111,139.612,בית יעקב מרכז,בני ברק,1,5,74
541391,137.844,ישיבת זכרון דוד,בני ברק,0,8,96


### Compare cities

In [20]:
yishuv_analysis = defaultdict(dict)

for school_yishuv_name in schools_df.school_yishuv_name.unique():
    yishuv_df = schools_df.loc[schools_df.school_yishuv_name == school_yishuv_name]
    df_2019_2020 = yishuv_df.loc[(yishuv_df.accident_timestamp >= pd.Timestamp('2019-06-01')) & (yishuv_df.accident_timestamp <= pd.Timestamp('2020-05-31'))]
    killed = df_2019_2020.loc[df_2019_2020.injury_severity_hebrew == 'הרוג'].inv_unique_id.nunique()
    severe_injured = df_2019_2020.loc[df_2019_2020.injury_severity_hebrew == 'פצוע קשה'].inv_unique_id.nunique()
    light_injured = df_2019_2020.loc[df_2019_2020.injury_severity_hebrew == 'פצוע קל'].inv_unique_id.nunique()
    score = (killed * killed_weight + severe_injured * severe_weight  + light_injured * light_weight) * (killed + severe_injured + light_injured)
    yishuv_analysis[school_yishuv_name]['score'] = score
    yishuv_analysis[school_yishuv_name]['school_yishuv_name'] = school_yishuv_name
    yishuv_analysis[school_yishuv_name]['killed'] = killed
    yishuv_analysis[school_yishuv_name]['severe_injured'] = severe_injured
    yishuv_analysis[school_yishuv_name]['light_injured'] = light_injured
yishuv_analysis = pd.DataFrame(yishuv_analysis).T


In [21]:
yishuv_analysis.sort_values('score', ascending=False).iloc[0:40]

,score,school_yishuv_name,killed,severe_injured,light_injured
ירושלים,255.861,ירושלים,2,5,89
תל אביב - יפו,200.397,תל אביב - יפו,2,5,70
אשדוד,86.7322,אשדוד,2,2,37
בני ברק,56.381,בני ברק,0,4,72
לוד,19.2778,לוד,1,3,11
נתניה,13.1159,נתניה,0,2,34
רמת גן,11.2365,רמת גן,0,2,30
פתח תקווה,10.5786,פתח תקווה,0,1,40
רמלה,10.2833,רמלה,1,2,6
מודיעין עילית,9.17808,מודיעין עילית,1,1,7


In [22]:
yishuv_analysis.sort_values('score', ascending=False)

,score,school_yishuv_name,killed,severe_injured,light_injured
ירושלים,255.861,ירושלים,2,5,89
תל אביב - יפו,200.397,תל אביב - יפו,2,5,70
אשדוד,86.7322,אשדוד,2,2,37
בני ברק,56.381,בני ברק,0,4,72
לוד,19.2778,לוד,1,3,11
...,...,...,...,...,...
נשר,0,נשר,0,0,0
יפיע,0,יפיע,0,0,0
כוכב יאיר,0,כוכב יאיר,0,0,0
תראבין א-צאנע(,0,תראבין א-צאנע(,0,0,0


In [23]:
yishuv_analysis[yishuv_analysis.score > 0].sort_values('score', ascending=False)

,score,school_yishuv_name,killed,severe_injured,light_injured
ירושלים,255.861,ירושלים,2,5,89
תל אביב - יפו,200.397,תל אביב - יפו,2,5,70
אשדוד,86.7322,אשדוד,2,2,37
בני ברק,56.381,בני ברק,0,4,72
לוד,19.2778,לוד,1,3,11
...,...,...,...,...,...
שערי תקווה,0.00329772,שערי תקווה,0,0,1
גבעת זאב,0.00329772,גבעת זאב,0,0,1
בנימינה-גבעת ע,0.00329772,בנימינה-גבעת ע,0,0,1
קרית טבעון,0.00329772,קרית טבעון,0,0,1


### load yishuv data

In [47]:
yishuv_data = pd.read_csv(os.path.join('static/data/schools/yishuv_data.csv'), index_col=None, header=0)

In [48]:
yishuv_data.columns

Index(['שם  הרשות', 'סמל הרשות', 'מחוז ', 'מעמד מוניציפלי',
       'סה"כ אוכלוסייה בסוף השנה (אלפים)', 'בני 4-0', 'בני 9-5', 'בני 14-10',
       'בני 19-15', 'בני 29-20', 'בני 44-30', 'בני 59-45', 'בני 64-60',
       'בני 65 ומעלה', 'בני 17-0', 'בני 75 ומעלה', 'סה"כ תלמידים',
       'בבתי ספר יסודיים (כולל חינוך מיוחד)', 'בבתי ספר על-יסודיים',
       'בחטיבות ביניים', 'בבתי ספר תיכוניים'],
      dtype='object')

In [49]:
yishuv_data['5_19_population'] = ( 
                                 yishuv_data['בני 9-5'] + \
                                 yishuv_data['בני 14-10'] + \
                                 yishuv_data['בני 19-15'] ) * 1000 * yishuv_data['סה"כ אוכלוסייה בסוף השנה (אלפים)'] / 100


In [50]:
yishuv_data.set_index('שם  הרשות', inplace=True)

In [51]:
yishuv_data.index = yishuv_data.index.str.strip().str.replace('-', ' ').str.replace('  ', ' ').str.replace('  ', ' ').str.replace('יי', 'י').str.replace('"', '').str.replace("'", '').str.replace('*','').str.replace('מכבים רעות','מכבים')



In [52]:
yishuv_analysis.index = yishuv_analysis.index.str.strip().str.replace('-', ' ').str.replace('  ', ' ').str.replace('  ', ' ').str.replace('יי', 'י').str.replace('"', '').str.replace("'", '').str.replace('*','').str.replace('מכבים רעות','מכבים')


In [53]:
merged = yishuv_analysis.join(yishuv_data)

In [63]:
merged['normalized_score'] = merged['score'] / merged['5_19_population']

In [64]:
merged.dropna(subset=['normalized_score'], inplace=True)

In [65]:
merged.sort_values('5_19_population', ascending=False).loc[:,['score', 'normalized_score', 'killed', 'severe_injured', 'light_injured', '5_19_population']].to_csv('results_2020_yishuv_table.csv')
                                                       